In [ ]:
import pandas as pd
import numpy as np
import re

# =========================
# 1. Read data
# =========================
ccle_drug = pd.read_csv(
    "./data/CCLE25Q3/Drug_sensitivity_AUC_(CTD^2)_subsetted.csv"
)

tcga_drug = pd.read_csv(
    "./data/2016_tcga_drug.csv"
)

# =========================
# 2. Extract drug names
# =========================
tcga_drug_name = np.array(tcga_drug.drug_name[1:].unique(), dtype=str)
ccle_drug_name = np.unique([re.sub(r"\s*\(.*\)", "", n) for n in ccle_drug.columns[1:]])[1:]

# =========================
# 3. Upper-case normalization
# =========================
tcga_up = np.char.upper(tcga_drug_name)
ccle_up = np.char.upper(ccle_drug_name)

# =========================
# 4. Find intersection
# =========================
common_drug_name = np.intersect1d(tcga_up, ccle_up)

print(f"Common drugs found: {len(common_drug_name)}")
print(common_drug_name)

# =========================
# 5. Filter TCGA table (row-based)
# =========================
tcga_drug["drug_name_upper"] = (
    tcga_drug["drug_name"]
    .astype(str)
    .str.upper()
)

tcga_drug_common = tcga_drug[
    tcga_drug["drug_name_upper"].isin(common_drug_name)
].copy()

print(f"TCGA rows after filtering: {tcga_drug_common.shape}")

# =========================
# 6. Filter CCLE table (column-based)
# =========================
# Normalize CCLE column names
ccle_cols = np.array(ccle_drug.columns, dtype=str)

ccle_cols_upper = np.char.upper([
    re.sub(r"\s*\(.*\)", "", c) for c in ccle_cols
])

# Mask for drugs in intersection
keep_mask = np.isin(ccle_cols_upper, common_drug_name)

# Always keep first column (cell line ID)
keep_mask[0] = True

ccle_drug_common = ccle_drug.loc[:, keep_mask].copy()

print(f"✅ CCLE columns after filtering: {ccle_drug_common.shape}")

# =========================
# 7. Sanity check
# =========================
print("\n🔎 Drugs in filtered TCGA:")
print(np.sort(tcga_drug_common["drug_name_upper"].unique()))

print("\n🔎 Drugs in filtered CCLE:")
print(np.sort(np.unique(ccle_cols_upper[keep_mask][1:])))

# =========================
# 8. (Optional) Save outputs
# =========================
tcga_drug_common.to_csv(
    "./tcga_drug_common_with_ccle.csv",
    index=False
)

ccle_drug_common.to_csv(
    "./ccle_drug_common_with_tcga.csv",
    index=False
)


✅ Common drugs found: 31
['CABOZANTINIB' 'CARBOPLATIN' 'CYCLOPHOSPHAMIDE' 'DABRAFENIB'
 'DACARBAZINE' 'DASATINIB' 'DEXAMETHASONE' 'DOCETAXEL' 'DOXORUBICIN'
 'ERLOTINIB' 'ETOPOSIDE' 'FLUOROURACIL' 'FULVESTRANT' 'GEFITINIB'
 'GEMCITABINE' 'IFOSFAMIDE' 'IMATINIB' 'LAPATINIB' 'METHOTREXATE'
 'MITOMYCIN' 'PACLITAXEL' 'PAZOPANIB' 'PROCARBAZINE' 'REGORAFENIB'
 'RIGOSERTIB' 'TEMSIROLIMUS' 'THALIDOMIDE' 'TOPOTECAN' 'TRAMETINIB'
 'VINCRISTINE' 'VORINOSTAT']
✅ TCGA rows after filtering: (1286, 15)
✅ CCLE columns after filtering: (840, 32)

🔎 Drugs in filtered TCGA:
['CABOZANTINIB' 'CARBOPLATIN' 'CYCLOPHOSPHAMIDE' 'DABRAFENIB'
 'DACARBAZINE' 'DASATINIB' 'DEXAMETHASONE' 'DOCETAXEL' 'DOXORUBICIN'
 'ERLOTINIB' 'ETOPOSIDE' 'FLUOROURACIL' 'FULVESTRANT' 'GEFITINIB'
 'GEMCITABINE' 'IFOSFAMIDE' 'IMATINIB' 'LAPATINIB' 'METHOTREXATE'
 'MITOMYCIN' 'PACLITAXEL' 'PAZOPANIB' 'PROCARBAZINE' 'REGORAFENIB'
 'RIGOSERTIB' 'TEMSIROLIMUS' 'THALIDOMIDE' 'TOPOTECAN' 'TRAMETINIB'
 'VINCRISTINE' 'VORINOSTAT']

🔎 Drugs in 